In [17]:
import os
import torch
from tqdm import tqdm
import pandas as pd

In [34]:
DATA_ROOT = '../../dataset'
TEST_DATASET_PATH = os.path.join(DATA_ROOT, 'test')

INFERENCE_SIZE = 1024
AUGMENT = True

IMAGES = []
for image_name in os.listdir(TEST_DATASET_PATH):
    if not image_name.startswith('._'):
        IMAGES.append(os.path.join(TEST_DATASET_PATH, image_name))

IMAGES.sort()

PROJECT_NAME = 'bc_lv2-object_detection'
EXPERIMENT_NAME = 'yolov7w6_f4_default'

model = torch.hub.load(
    repo_or_dir = './',
    model = 'custom',
    source='local' 
    )

model.conf = 0.001  # NMS confidence threshold
model.iou = 0.6  # NMS IoU threshold

# model.agnostic = False  # NMS class-agnostic
# model.multi_label = False  # NMS multiple labels per box
# model.classes = None  # (optional list) filter by class, i.e. = [0, 15, 16] for COCO persons, cats and dogs
# model.max_det = 1000  # maximum number of detections per image
# model.amp = False  # Automatic Mixed Precision (AMP) inference
IMAGES[0]


                 from  n    params  module                                  arguments                     
  0                -1  1         0  models.common.ReOrg                     []                            
  1                -1  1      7040  models.common.Conv                      [12, 64, 3, 1]                
  2                -1  1     73984  models.common.Conv                      [64, 128, 3, 2]               
  3                -1  1      8320  models.common.Conv                      [128, 64, 1, 1]               
  4                -2  1      8320  models.common.Conv                      [128, 64, 1, 1]               
  5                -1  1     36992  models.common.Conv                      [64, 64, 3, 1]                
  6                -1  1     36992  models.common.Conv                      [64, 64, 3, 1]                
  7                -1  1     36992  models.common.Conv                      [64, 64, 3, 1]                
  8                -1  1     36992  

Adding autoShape... 


'../../dataset/test/0000.jpg'

In [35]:
prediction_strings = []
file_names = []

for i in tqdm(range(len(IMAGES))):
    prediction_string = ''
    
    output = model(IMAGES[i], size=INFERENCE_SIZE, augment=AUGMENT)
    for bbox in output.pandas().xyxy[0].values:
        xmin, ymin, xmax, ymax, conf, cls, _ = bbox
        prediction_string += f'{cls} {conf} {xmin} {ymin} {xmax} {ymax} '
    
    path_split = IMAGES[i].split('/')
    image_path = f'{path_split[-2]}/{path_split[-1]}'

    prediction_strings.append(prediction_string)
    file_names.append(image_path)

submission = pd.DataFrame()
submission['PredictionString'] = prediction_strings
submission['image_id'] = file_names
submission.head()

100%|██████████| 4871/4871 [10:09<00:00,  7.99it/s]


,PredictionString,image_id
0,7 0.990167498588562 603.7650146484375 515.6267...,test/0000.jpg
1,5 0.9648200273513794 341.5235595703125 249.747...,test/0001.jpg
2,1 0.9482503533363342 75.65573120117188 269.929...,test/0002.jpg
3,9 0.9568088054656982 147.60311889648438 253.80...,test/0003.jpg
4,1 0.9006543159484863 188.72637939453125 253.48...,test/0004.jpg


In [36]:
submission.to_csv(os.path.join(f'./{PROJECT_NAME}/{EXPERIMENT_NAME}', f'{EXPERIMENT_NAME}.csv'), index=None)

In [37]:
df = pd.read_csv(os.path.join(f'./{PROJECT_NAME}/{EXPERIMENT_NAME}',f'{EXPERIMENT_NAME}.csv'))
df.loc[1]['PredictionString']

'5 0.9648200273513794 341.5235595703125 249.74754333496094 751.9813232421875 695.4154052734375 5 0.8307164907455444 130.7783660888672 0.0 493.10107421875 272.19134521484375 0 0.5419055819511414 764.978271484375 664.66943359375 899.826171875 982.52978515625 6 0.5305799245834351 764.0183715820312 657.180419921875 896.8715209960938 981.0755615234375 5 0.0075094010680913925 131.86627197265625 84.41046905517578 298.0430908203125 274.71759033203125 0 0.006969708018004894 664.7298583984375 207.69378662109375 1015.29833984375 437.85369873046875 5 0.006072625983506441 129.64529418945312 0.120269775390625 271.1781921386719 266.25311279296875 0 0.004922586493194103 764.9520263671875 639.9286499023438 830.8016357421875 669.7605590820312 5 0.0033240464981645346 131.5847930908203 0.1239471435546875 494.14398193359375 461.95269775390625 5 0.002658664947375655 124.81582641601562 0.0 345.1543884277344 282.1531982421875 1 0.0024340800009667873 742.3494262695312 663.038818359375 903.4636840820312 986.960